In [1]:
import pandas as pd
import random
from tqdm import tqdm
import numpy as np
from scipy.sparse import csr_matrix
import scipy.sparse as sp 
import lightgbm as lgb
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,MultiLabelBinarizer
from itertools import chain

In [2]:
KEYWORDSTAGS = ['feedid','manual_keyword_list','machine_keyword_list','manual_tag_list', 'machine_tag_list']

In [3]:
origin_feed = pd.read_csv('../data/wechat_algo_data1/feed_info.csv')[KEYWORDSTAGS]

In [4]:
for i in KEYWORDSTAGS[1:]:
    origin_feed[i] = origin_feed[i].fillna(0)

In [5]:
origin_feed

,feedid,manual_keyword_list,machine_keyword_list,manual_tag_list,machine_tag_list
0,43549,15506;7715;17582,26334;219;25209;7715;18541,81;269;159;6,269 0.8525666;81 0.8525666;8 1.1e-07;306 0.0;2...
1,77432,8199;18322;4243,24078;19924,194;267;159;6,267 0.99293476;194 0.99293476
2,12921,0,11696;6926,202;23;160;6,23 0.64771646;202 0.64771646
3,34538,5454;1197;3727,0,328;13;159;6,267 0.21298289;191 0.21298289;8 0.34298885;306...
4,75414,17083;4123,8058;219;21639;15621;25209,81;269;159;6,269 0.86163938;81 0.86163938
...,...,...,...,...,...
106439,104384,10254;10844;1197,0,202;23;159;6,271 0.57400888;24 0.57400888
106440,91514,0,0,9;6,14 0.68890297;61 0.68890297;8 4e-08;306 0.0;20...
106441,69742,11411;18009;1312,0,179;262;159;6,262 0.81548262;179 0.81548262;8 0.16021954;306...
106442,61540,3212;17707;17582,0,96;269;159;6,269 0.53156692;82 0.53156692


In [ ]:
#用于处理machine_tag_list
# def machine_tag_list_deal(df):
#     res = []
#     for elem in df:
#         if isinstance(elem,int):
#             res.append(0)
#         else:
#             tmp = elem.split(";")
#             n = len(tmp)
#             ans = 0
#             for j in tmp:
#                 tagp = list(map(float,j.split(' ')))
#                 ans += tagp[0] * tagp[1]
#             res.append(ans / n)
#     return res 

In [ ]:
def func(df,idx):
    res = []
    for x in df[idx]:
        if x != 0:
            res.append(list(map(int,x.split(';'))))
        else:
            res.append([0])
    print(len(set(list(chain.from_iterable(res)))))
    mlb = MultiLabelBinarizer(sparse_output=True)
    #print(list(mlb.classes))
    return mlb.fit_transform(res)

In [ ]:
res = func(origin_feed,'manual_keyword_list')
print(res.shape)

In [ ]:
res

In [ ]:
one_hot = pd.read_csv('feed_embeddings_100_one_hot.csv')

In [ ]:
one_hot

In [ ]:
k_means = pd.read_csv('K_means_embeddings.csv')

In [ ]:
k_means

In [ ]:
res = k_means['feedid'].values

In [ ]:
one_hot['feedid']=res 

In [ ]:
one_hot

In [ ]:
c = one_hot.pop('feedid')

In [ ]:
c 

In [ ]:
one_hot.insert(0,'feedid',c)

In [ ]:
one_hot

In [ ]:
one_hot.to_csv('feed_embeddings_100_k_means.csv',index=False)

In [7]:
len(set(origin_feed['machine_keyword_list'].values))

54801

In [2]:
tag_data = pd.read_csv("feed_info_tag.csv")[['feedid','tag_1','tag_2']]

In [3]:
tag_data

,feedid,tag_1,tag_2
0,43549,81,269
1,77432,194,267
2,12921,202,23
3,34538,328,13
4,75414,81,269
...,...,...,...
106439,104384,202,23
106440,91514,9,6
106441,69742,179,262
106442,61540,96,269


In [4]:
len(set(tag_data['tag_1']))

242

In [5]:
len(set(tag_data['tag_2']))

42

In [8]:
len(set(list(tag_data['tag_1'].values) + list(tag_data['tag_2'].values)))

270

In [10]:
tag_data[['tag_1','tag_2']].values

array([[ 81, 269],
       [194, 267],
       [202,  23],
       ...,
       [179, 262],
       [ 96, 269],
       [202,  23]])

In [11]:
multi_hot_encoder = MultiLabelBinarizer(sparse_output=True)
all_encoded = tag_data[['tag_1','tag_2']].values
multi_hot_encoded = multi_hot_encoder.fit_transform(all_encoded)

In [12]:
multi_hot_encoded

<106444x270 sparse matrix of type '<class 'numpy.int64'>'
	with 211871 stored elements in Compressed Sparse Row format>

In [13]:
tmp = pd.DataFrame(multi_hot_encoded.todense())

In [14]:
tmp

,0,1,2,3,4,5,6,7,8,9,...,260,261,262,263,264,265,266,267,268,269
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106439,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
106440,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
106441,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
106442,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
tmp['feedid']=tag_data['feedid'].values

In [16]:
tmp

,0,1,2,3,4,5,6,7,8,9,...,261,262,263,264,265,266,267,268,269,feedid
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,43549
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,77432
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12921
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,34538
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,75414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106439,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,104384
106440,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,91514
106441,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,69742
106442,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,61540


In [17]:
tag_data.to_csv('two_cols_tags.csv',index=False)

In [18]:
idxs = ['tag_' + str(i) + '_fea' for i in range(270)] + ['feedid']
tmp.columns = idxs

In [19]:
tmp 

,tag_0_fea,tag_1_fea,tag_2_fea,tag_3_fea,tag_4_fea,tag_5_fea,tag_6_fea,tag_7_fea,tag_8_fea,tag_9_fea,...,tag_261_fea,tag_262_fea,tag_263_fea,tag_264_fea,tag_265_fea,tag_266_fea,tag_267_fea,tag_268_fea,tag_269_fea,feedid
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,43549
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,77432
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12921
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,34538
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,75414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106439,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,104384
106440,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,91514
106441,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,69742
106442,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,61540


In [23]:
c = tmp.pop('feedid')

In [24]:
tmp.insert(0,'feedid',c)

In [25]:
tmp 

,feedid,tag_0_fea,tag_1_fea,tag_2_fea,tag_3_fea,tag_4_fea,tag_5_fea,tag_6_fea,tag_7_fea,tag_8_fea,...,tag_260_fea,tag_261_fea,tag_262_fea,tag_263_fea,tag_264_fea,tag_265_fea,tag_266_fea,tag_267_fea,tag_268_fea,tag_269_fea
0,43549,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,77432,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,12921,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,34538,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,75414,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106439,104384,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
106440,91514,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
106441,69742,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
106442,61540,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
tmp.to_csv('270_cols_tags.csv',index=False)

In [6]:
origin_feed['manual_tag_list'].values

array(['81;269;159;6', '194;267;159;6', '202;23;160;6', ...,
       '179;262;159;6', '96;269;159;6', '202;23;160;6'], dtype=object)

In [10]:
def new_func(data):
    res = []
    for d in data:
        if isinstance(d,str):
            tmp = list(map(int,d.split(';')))
            if len(tmp) == 4:
                res.append(tmp)
            else:
                diff = 4 - len(tmp)
                tmp = tmp + [0 for _ in range(diff)]
                res.append(tmp)
            res.append(tmp)
        else:
            res.append([0])
    return res 

In [11]:
res = new_func(origin_feed['manual_tag_list'].values)

In [12]:
res

[[81, 269, 159, 6],
 [81, 269, 159, 6],
 [194, 267, 159, 6],
 [194, 267, 159, 6],
 [202, 23, 160, 6],
 [202, 23, 160, 6],
 [328, 13, 159, 6],
 [328, 13, 159, 6],
 [81, 269, 159, 6],
 [81, 269, 159, 6],
 [45, 239, 159, 169],
 [45, 239, 159, 169],
 [9, 6, 0, 0],
 [9, 6, 0, 0],
 [9, 6, 0, 0],
 [9, 6, 0, 0],
 [9, 6, 0, 0],
 [9, 6, 0, 0],
 [81, 269, 159, 6],
 [81, 269, 159, 6],
 [340, 235, 159, 169],
 [340, 235, 159, 169],
 [205, 23, 160, 6],
 [205, 23, 160, 6],
 [9, 6, 0, 0],
 [9, 6, 0, 0],
 [61, 14, 159, 6],
 [61, 14, 159, 6],
 [219, 6, 0, 0],
 [219, 6, 0, 0],
 [81, 269, 159, 6],
 [81, 269, 159, 6],
 [219, 6, 0, 0],
 [219, 6, 0, 0],
 [291, 234, 159, 6],
 [291, 234, 159, 6],
 [170, 239, 159, 169],
 [170, 239, 159, 169],
 [9, 6, 0, 0],
 [9, 6, 0, 0],
 [32, 17, 159, 6],
 [32, 17, 159, 6],
 [328, 13, 212, 159, 6],
 [328, 13, 212, 159, 6],
 [219, 6, 0, 0],
 [219, 6, 0, 0],
 [340, 235, 159, 6],
 [340, 235, 159, 6],
 [194, 267, 159, 212, 6],
 [194, 267, 159, 212, 6],
 [332, 16, 159, 6],
 [332, 1

In [17]:
maxnum = 0
for i in res:
    maxnum = max(maxnum,len(i))

In [18]:
maxnum

11

In [21]:
multi_hot_encoder = MultiLabelBinarizer(sparse_output=True)
all_encoded = res 
multi_hot_encoded = multi_hot_encoder.fit_transform(all_encoded)

In [22]:
multi_hot_encoded

<106444x350 sparse matrix of type '<class 'numpy.int64'>'
	with 398196 stored elements in Compressed Sparse Row format>

In [29]:
store_data = pd.DataFrame(multi_hot_encoded.todense()).reset_index()

In [30]:
store_data['index'] = origin_feed['feedid']

In [31]:
store_data

,index,0,1,2,3,4,5,6,7,8,...,340,341,342,343,344,345,346,347,348,349
0,43549,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,77432,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,12921,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,34538,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,75414,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106439,104384,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
106440,91514,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
106441,69742,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
106442,61540,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
store_data.columns = ['feedid'] + ['feed_tag_' + str(j) for j in range(350)]

In [35]:
store_data

,feedid,feed_tag_0,feed_tag_1,feed_tag_2,feed_tag_3,feed_tag_4,feed_tag_5,feed_tag_6,feed_tag_7,feed_tag_8,...,feed_tag_340,feed_tag_341,feed_tag_342,feed_tag_343,feed_tag_344,feed_tag_345,feed_tag_346,feed_tag_347,feed_tag_348,feed_tag_349
0,43549,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,77432,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,12921,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,34538,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,75414,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106439,104384,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
106440,91514,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
106441,69742,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
106442,61540,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
store_data.to_csv('350_cols_tags.csv',index=False)

In [38]:
store_data

,feedid,feed_tag_0,feed_tag_1,feed_tag_2,feed_tag_3,feed_tag_4,feed_tag_5,feed_tag_6,feed_tag_7,feed_tag_8,...,feed_tag_340,feed_tag_341,feed_tag_342,feed_tag_343,feed_tag_344,feed_tag_345,feed_tag_346,feed_tag_347,feed_tag_348,feed_tag_349
0,43549,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,77432,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,12921,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,34538,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,75414,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106439,104384,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
106440,91514,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
106441,69742,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
106442,61540,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
